Q1. Embedding the query

In [9]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [42]:
from fastembed import TextEmbedding
import numpy as np

embedder = TextEmbedding(model_name=model_handle)
Q1_question = "I just discovered the course. Can I join now?"
# Embed the query
embedding = next(embedder.embed([Q1_question]))
embedding = np.array(embedding)
print(embedding.shape)  # Should print (512,)

(384,)


In [11]:
embedding.min()

np.float64(-0.11726373885183883)

In [12]:
import numpy as np
np.linalg.norm(embedding)

np.float64(1.0)

In [13]:
q = embedding
q.dot(q)

np.float64(1.0000000000000002)

Q2. Cosine similarity with another vector

In [14]:
doc = 'Can I still join the course after the start date?'
# Embed the document
doc_embedding = next(embedder.embed([doc]))
doc_embedding = np.array(doc_embedding)

# Compute cosine similarity
cosine_similarity = np.dot(embedding, doc_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(doc_embedding))
cosine_similarity

np.float64(0.9008528895674548)

Q3. Ranking by cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
# Compute embeddings for the 'text' field of each document
doc_embeddings = [np.array(next(embedder.embed([doc['text']]))) for doc in documents]

# Compute cosine similarity between the query vector 'embedding' and each document embedding
cosine_similarities = [
    float(np.dot(embedding, doc_emb) / (np.linalg.norm(embedding) * np.linalg.norm(doc_emb)))
    for doc_emb in doc_embeddings
]

cosine_similarities


[0.7629684696540238,
 0.8182378150042889,
 0.8085397398734189,
 0.7133079015686243,
 0.7304499234333598]

In [17]:
most_similar_idx = int(np.argmax(cosine_similarities))
most_similar_idx

1

Q4. Ranking by cosine, version two

In [18]:
full_text_embeddings = [np.array(next(embedder.embed([doc['question'] + " " + doc['text']]))) for doc in documents]

In [20]:
# Normalize all full_text_embeddings
full_text_embeddings_norm = [vec / np.linalg.norm(vec) for vec in full_text_embeddings]

# Normalize the query embedding
embedding_norm = embedding / np.linalg.norm(embedding)

# Compute cosine similarities with the normalized query embedding
cosine_similarities_full_text = [float(np.dot(embedding_norm, vec_norm)) for vec_norm in full_text_embeddings_norm]

cosine_similarities_full_text

[0.8514543236908068,
 0.8436594159113068,
 0.840828704850256,
 0.7755157969663908,
 0.8086007795043938]

In [22]:
# Find the index of the highest scoring document
best_idx = int(np.argmax(cosine_similarities_full_text))

# Show the highest cosine similarity score and the corresponding document
print("Best matching document index:", best_idx)
print("Highest cosine similarity:", cosine_similarities_full_text[best_idx])
print("Best matching document text:", documents[best_idx]['text'])

Best matching document index: 0
Highest cosine similarity: 0.8514543236908068
Best matching document text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Q5. Selecting the embedding model

In [31]:
min_dim = min(model["dim"] for model in TextEmbedding.list_supported_models())
print("Smallest embedding dimensionality in fastembed models:", min_dim)

Smallest embedding dimensionality in fastembed models: 384


Q6. Indexing with qdrant

In [43]:
model_handle = "BAAI/bge-small-en"
embedder = TextEmbedding(model_name=model_handle)

# Embed the query   
embedding = next(embedder.embed([Q1_question]))
# Convert to numpy array"]))
embedding = np.array(embedding)
print(embedding.shape) 

(384,)


In [37]:

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [40]:
from qdrant_client.http import models
from qdrant_client import QdrantClient
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

# Use the embedding dimension from the normalized embedding
EMBEDDING_DIMENSIONALITY  = embedding.shape[0]  # Should be 384 for BAAI/bge-small-en


# Create the collection if it doesn't exist
collection_name = "llm-zoomcamp-hw2-vector-search"
existing_collections = [col.name for col in client.get_collections().collections]
if collection_name not in existing_collections:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=EMBEDDING_DIMENSIONALITY,
            distance=models.Distance.COSINE
        )
    )
# Prepare normalized embeddings using both question and answer fields
full_text_embeddings_norm = [
    vec / np.linalg.norm(vec) for vec in [
        np.array(next(embedder.embed([doc['question'] + " " + doc['text']])))
        for doc in documents
    ]
]
# Prepare points for upsert
points = []
for idx, vec in enumerate(full_text_embeddings_norm):
    point = models.PointStruct(
        id=idx,
        vector=vec.tolist(),
        payload={
            "text": documents[idx]['text'],
            "question": documents[idx]['question'],
            "section": documents[idx]['section'],
            "course": documents[idx]['course']
        }
    )
    points.append(point)

# Upsert points into Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [45]:
def search(query, limit=1):
	results = client.query_points(
		collection_name=collection_name,
		query=models.Document(
			text=query,
			model=model_handle 
		),
		limit=limit,
		with_payload=True
	)
	return results

result = search(Q1_question)
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'question': 'The course has already started. Can I still join it?', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])